# **Pokémon Diffusion<a id="top"></a>**

> #### `02-Diffusion-Model-Architecture.ipynb`

<i><small>**Alumno:** Alejandro Pequeño Lizcano<br>Última actualización: 08/04/2024</small></i></div>


El modelo de difusión se compone de dos partes: el **modelo en sí** y el proceso de difusión. Durante este notebook, se explicará la arquitectura del modelo de difusión y cómo se ha implementado en este proyecto. El modelo, es la red neuronal que se encarga de aprender a predecir el ruido en cada paso de difusión. El proceso de difusión es un proceso iterativo que se encarga de aplicar el bloque de difusión a la imagen original o a la imagen con ruido en cada paso de difusión. El proceso de difusión se repite un número fijo de veces para generar una imagen nueva con el ruido añadido.

Este modelo usado para la predicción del ruido, que aprenderá posteriormente a quitarlo de manera iterativa hasta producir una imagen nueva. Según muchos papers, se podría ser cualquier red neuronal, ya que no existe una arquitectura específica y depende del conjunto de datos con el que se entrene, no obstante, la más usada para la síntesis de imágenes y la que se ha usado en este proyecto es la arquitectura **U-Net** por sus características de recuperación de la información manteniendo la dimensionalidad de la imagen.

Esta arquitectura se caracteriza por tener una parte de codificación y una parte de decodificación que se conectan entre sí. Además, cada bloque de decodificación se conecta con el bloque de codificación correspondiente mediante una operación de concatenación. La arquitectura **U-Net** se caracteriza por tener una arquitectura simétrica, es decir, la parte de codificación y la parte de decodificación tienen la misma estructura.

Para construir el modelo, nos apoyaremos en la función de `build_u_net` que se encargará de construir la arquitectura de la red neuronal. Esta función, a su vez se apoyará en la función `diffusion_block` que se encargará de construir los bloques de codificación y decodificación de la red añadiendo la información temporal y la etiqueta de la imagen a generar, en nuestro caso, el tipo del Pokémon a generar.

- `diffusion_block()`: El bloque de difusión contiene tres parámetros:

  - `x_parameter`: es el tensor de entrada que contiene la imagen original o la imagen con ruido en cada paso de difusión.
  - `time_parameter` es el tensor que indica el paso de difusión en el que nos encontramos. Se usa para calcular el valor de **$\beta$** según el _scheduler_ que hayamos elegido.
  - `label_parameter` es el tensor que contiene la etiqueta de la imagen a generar. En nuestro caso, el tipo del Pokémon a generar.

  Dentro del bloque de difusión, se aplican transformaciones a cada uno de estos parámetros, lo que puede incluir capas densas, normalización y activación ReLU. Estas transformaciones capturan las relaciones y dependencias entre los diferentes aspectos de la entrada (imagen y tiempo). Finalmente, se calcula la imagen nueva con el ruido añadido.

- El proceso de difusión utiliza una arquitectura de tipo **U-Net** modificada con bloques de difsuión que toman en cuenta la imagen, su etiqueta y el tensor tiempo. Posteriormente, se realizan operaciones de convolución y pooling para reducir la resolución de la imagen mientras se procesa la información temporal. Luego, se realiza un proceso de decodificación utilizando operaciones de upsampling y concatenación para generar una imagen de salida que tiene la misma resolución que la imagen de entrada. Después de este proceso, se añade una capa **MLP** para procesar la información temporal y generar una imagen de salida. Finalmente, se devuelve la imagen de salida.

---

<i><small>**Más infromación** sobre el porqué matemático de la función de pérdida, aunque ya explicado, se puede encontrar en el paper [Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2006.11239) y una explicación más clara en la página [Diffusion Model Clearly Explained!](https://medium.com/@steinsfu/diffusion-model-clearly-explained-cd331bd41166).

><span style="color: red; font-size: 1.5em;">&#9888;</span> **NOTA:** El proceso matemático para llegar a esta fórmula es muy complejo para explicarlo en este notebook. Sin embargo, en el futuro informe se explicará con más detalle.
</small></i>


# 0. Imports

Una vez introducido el proyecto, se importan las librerías necesarias para el desarrollo de este apartado.

---


In [1]:
# Import necessary libraries
# =====================================================================

# Import libraries for data preprocessing
import configparser
import tensorflow as tf

# Import src code
from src.model.build_unet import *
from src.utils.utils import PROJECT_DIR

2024-05-16 15:02:28.580584: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-16 15:02:28.925769: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'src.model.build_unet'

In [2]:
# Hyperparameters
# =====================================================================
config = configparser.ConfigParser()
config.read(PROJECT_DIR / "config.ini")
config_hp = config["hyperparameters"]

IMG_SIZE = int(config_hp["img_size"])
NUM_CLASSES = int(config_hp["num_classes"])

# Modelo de Difusión

En este apartado, se construirá el modelo de difusión que se encargará de aprender a predecir el ruido en cada paso de difusión. Para ello, como se ha comentado con anterioridad, se construirá una red neuronal basada en la arquitectura **U-Net** que se encargará de generar una imagen nueva con el ruido añadido.

El motivo de la función de pérdida, se explicará en el siguiente notebook, pero se basa en la función de pérdida **MSE** que se encargará de calcular la diferencia entre la imagen original y la imagen generada. Esta función de pérdida se encargará de minimizar el error entre ambas imágenes.


In [3]:
# Create the model
# =====================================================================
model = build_unet(IMG_SIZE, NUM_CLASSES)

# Compile the model
# =====================================================================
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss_fn = tf.keras.losses.MeanSquaredError()
model.compile(loss=loss_fn, optimizer=optimizer)

# Show the model summary
# =====================================================================
model.summary()

2024-05-07 22:17:01.040810: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-07 22:17:01.045322: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-07 22:17:01.045464: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 18)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │        608 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 32)        │         64 │ dense[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 32)        │          0 │ layer_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, 64, 64, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      2,112 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sinusoidal_time_em… │ (None, 32)        │          0 │ input_layer_2[0]… │
│ (SinusoidalTimeEmb… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 64, 64,    │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 64)        │          0 │ dense_2[0][0]     │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      2,112 │ sinusoidal_time_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 64, 64,    │          0 │ conv2d[0][0]      │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 1, 1, 64)  │          0 │ activation_3[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 64)        │          0 │ dense_1[0][0]     │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 64, 64,    │          0 │ activation_1[0][… │
│                     │ 64)               │            │ reshape_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 1, 1, 64)  │          0 │ activation_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 64, 64,    │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 64, 64,    │          0 │ multiply[0][0],   │
│                     │ 64)               │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 26,423,267 (100.80 MB)

 Trainable params: 26,423,267 (100.80 MB)

 Non-trainable params: 0 (0.00 B)

[BACK TO TOP](#top)
